In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

## Read Preprocessed.csv

In [3]:
df = pd.read_csv("Preprocessed.csv")

## Train-test split


In [4]:
X = df.drop(columns="AcceptedCmp6")
y = df["AcceptedCmp6"]
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)

##Scale

In [ ]:
scaler = MinMaxScaler()

X_train_scaled_array = scaler.fit_transform(X_train)
X_test_scaled_array = scaler.transform(X_test)

X_train_scaled= pd.DataFrame(X_train_scaled_array, columns=X_train.columns, index=X_train.index)
X_test_scaled= pd.DataFrame(X_test_scaled_array, columns=X_test.columns, index=X_test.index)

##Decision Tree

In [ ]:
tree = DecisionTreeClassifier( random_state = 42)
tree.fit(X_train, y_train)
print(f"Accuracy on training set: {tree.score(X_train, y_train)}")
y_pred=tree.predict(X_test)
print(f"Accuracy on test set: {metrics.accuracy_score(y_test, y_pred)}")

Accuracy on training set: 0.9934768427919113
Accuracy on test set: 0.80859375


# Define Profit Function

In [ ]:
call_cost = 3  # Cost per call in $
sale_revenue = 11  # Revenue per sale in $
false_positive_cost = call_cost  # Cost per False Positive in $

def calculate_profit(y_test_set, y_predict_proba_set, threshold):
    # Adjust predictions based on threshold
    y_predict_adjusted = (y_predict_proba_set >= threshold).astype(int)

    # Confusion Matrix: TN, FP, FN, TP
    tn, fp, fn, tp = confusion_matrix(y_test_set, y_predict_adjusted).ravel()
    #https://scikit-learn.org/1.5/modules/generated/sklearn.metrics.confusion_matrix.html

    # Calculate profit:
    profit_from_sales = tp * (sale_revenue - call_cost)
    loss_from_wasted_calls = fp * false_positive_cost
    total_profit = profit_from_sales - loss_from_wasted_calls
    return total_profit, tn, fp, fn, tp, threshold


# Define function for finding the best threshold

In [ ]:
def evaluate_thresholds(best_decision_tree_model, X_train, y_train, X_test, y_test):
    # Get probabilities for the positive class (class=1)
    y_train_prob = best_decision_tree_model.predict_proba(X_train)[:, 1]
    y_test_prob = best_decision_tree_model.predict_proba(X_test)[:, 1]

    thresholds = np.arange(0.1, 1.0, 0.05)  # Try different thresholds from 0.1 to 1.0
    best_profit = float('-inf') #If it's set to zero runtime error
    best_threshold = 0
    best_params = None

    for threshold in thresholds:
        # Calculating profit on training data
        profit, tn, fp, fn, tp, th = calculate_profit(y_train, y_train_prob, threshold)
        print(f"Threshold: {th:.2f} -> Profit: {profit:.2f}, TP: {tp}, FP: {fp}, FN {fn}, TN {tn} ,Threshold: {th:.2f} ")

        # Store the best threshold with maximum profit
        if profit > best_profit:
            best_profit = profit
            best_threshold = th
            best_params = (tn, fp, fn, tp, th)

    # Calculate profit for test data
    profit_test, tn_test, fp_test, fn_test, tp_test, th = calculate_profit(y_test, y_test_prob, best_threshold)

    # Calculate profit for train data
    profit_train, tn_train, fp_train, fn_train, tp_train, th = calculate_profit(y_train, y_train_prob, best_threshold)

    return {
        "Best Threshold": best_threshold,
        "Best profit on training set": profit_train,
        "Best profit on test set": profit_test,
        "Confusion matrix of training set": (tn_train, fp_train, fn_train, tp_train),
        "Confusion matrix of test set": (tn_test, fp_test, fn_test, tp_test)
    }


# Decision Tree without Tuning

In [ ]:
tree = DecisionTreeClassifier( random_state = 42)
tree.fit(X_train, y_train)
print(f"Accuracy on training set: {tree.score(X_train, y_train)}")
y_pred=tree.predict(X_test)
print(f"Accuracy on test set: {metrics.accuracy_score(y_test, y_pred)} \n")

evaluate_thresholds(tree, X_train, y_train, X_test, y_test)

Accuracy on training set: 0.9934768427919113
Accuracy on test set: 0.80859375 

Threshold: 0.10 -> Profit: 1794.00, TP: 228, FP: 10, FN 0, TN 1295 ,Threshold: 0.10 
Threshold: 0.15 -> Profit: 1794.00, TP: 228, FP: 10, FN 0, TN 1295 ,Threshold: 0.15 
Threshold: 0.20 -> Profit: 1794.00, TP: 228, FP: 10, FN 0, TN 1295 ,Threshold: 0.20 
Threshold: 0.25 -> Profit: 1794.00, TP: 228, FP: 10, FN 0, TN 1295 ,Threshold: 0.25 
Threshold: 0.30 -> Profit: 1794.00, TP: 228, FP: 10, FN 0, TN 1295 ,Threshold: 0.30 
Threshold: 0.35 -> Profit: 1794.00, TP: 228, FP: 10, FN 0, TN 1295 ,Threshold: 0.35 
Threshold: 0.40 -> Profit: 1794.00, TP: 228, FP: 10, FN 0, TN 1295 ,Threshold: 0.40 
Threshold: 0.45 -> Profit: 1794.00, TP: 228, FP: 10, FN 0, TN 1295 ,Threshold: 0.45 
Threshold: 0.50 -> Profit: 1744.00, TP: 218, FP: 0, FN 10, TN 1305 ,Threshold: 0.50 
Threshold: 0.55 -> Profit: 1744.00, TP: 218, FP: 0, FN 10, TN 1305 ,Threshold: 0.55 
Threshold: 0.60 -> Profit: 1744.00, TP: 218, FP: 0, FN 10, TN 1305 ,Th

{'Best Threshold': 0.1,
 'Best profit on training set': 1794,
 'Best profit on test set': 119,
 'Confusion matrix of training set': (1295, 10, 0, 228),
 'Confusion matrix of test set': (370, 59, 46, 37)}

# Decision Tree Tuned Based on F1

In [ ]:
decision_tree_tuned = DecisionTreeClassifier(random_state=42,class_weight = "balanced")
param_grid = {
    'max_depth': [2,3,5,6,7],
    'criterion': ['gini', 'entropy'],
    'min_samples_split': [2, 10, 20, 25],
    'min_samples_leaf': [1, 5, 10]
}
grid_search = GridSearchCV(decision_tree_tuned, param_grid, cv = 5 , scoring = 'f1')
grid_search.fit(X_train, y_train)

# Get the best parameters and model
best_decision_tree_model = grid_search.best_estimator_
print("Best Parameters:", grid_search.best_params_)

# Use the best model to make predictions on the test set
y_pred = best_decision_tree_model.predict(X_test)

training_score = best_decision_tree_model.score(X_train, y_train)
print("Accuracy on training set: ", training_score)
print(f"Accuracy on test set: {metrics.accuracy_score(y_test, y_pred)} \n")


evaluate_thresholds(best_decision_tree_model,  X_train, y_train, X_test, y_test)

Best Parameters: {'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 5, 'min_samples_split': 20}
Accuracy on training set:  0.8499673842139596
Accuracy on test set: 0.79296875 

Threshold: 0.10 -> Profit: -426.00, TP: 228, FP: 750, FN 0, TN 555 ,Threshold: 0.10 
Threshold: 0.15 -> Profit: -426.00, TP: 228, FP: 750, FN 0, TN 555 ,Threshold: 0.15 
Threshold: 0.20 -> Profit: -80.00, TP: 224, FP: 624, FN 4, TN 681 ,Threshold: 0.20 
Threshold: 0.25 -> Profit: -80.00, TP: 224, FP: 624, FN 4, TN 681 ,Threshold: 0.25 
Threshold: 0.30 -> Profit: 160.00, TP: 218, FP: 528, FN 10, TN 777 ,Threshold: 0.30 
Threshold: 0.35 -> Profit: 160.00, TP: 218, FP: 528, FN 10, TN 777 ,Threshold: 0.35 
Threshold: 0.40 -> Profit: 338.00, TP: 208, FP: 442, FN 20, TN 863 ,Threshold: 0.40 
Threshold: 0.45 -> Profit: 859.00, TP: 173, FP: 175, FN 55, TN 1130 ,Threshold: 0.45 
Threshold: 0.50 -> Profit: 859.00, TP: 173, FP: 175, FN 55, TN 1130 ,Threshold: 0.50 
Threshold: 0.55 -> Profit: 909.00, TP: 165, FP: 

{'Best Threshold': 0.7000000000000002,
 'Best profit on training set': 910,
 'Best profit on test set': 251,
 'Confusion matrix of training set': (1179, 126, 67, 161),
 'Confusion matrix of test set': (366, 63, 28, 55)}

# Logistic Regression without tuning


In [ ]:
logistic_regression = LogisticRegression(random_state = 42)
logistic_regression.fit(X_train, y_train)
y_pred = logistic_regression.predict(X_test)
print(f"Accuracy on training set: {logistic_regression.score(X_train, y_train)}")
print(f"Accuracy on test set: {metrics.accuracy_score(y_test, y_pred)}\n")

evaluate_thresholds(logistic_regression, X_train, y_train, X_test, y_test)




Accuracy on training set: 0.8545335942596216
Accuracy on test set: 0.837890625

Threshold: 0.10 -> Profit: -551.00, TP: 179, FP: 661, FN 49, TN 644 ,Threshold: 0.10 
Threshold: 0.15 -> Profit: -105.00, TP: 123, FP: 363, FN 105, TN 942 ,Threshold: 0.15 
Threshold: 0.20 -> Profit: 148.00, TP: 98, FP: 212, FN 130, TN 1093 ,Threshold: 0.20 
Threshold: 0.25 -> Profit: 229.00, TP: 77, FP: 129, FN 151, TN 1176 ,Threshold: 0.25 
Threshold: 0.30 -> Profit: 219.00, TP: 57, FP: 79, FN 171, TN 1226 ,Threshold: 0.30 
Threshold: 0.35 -> Profit: 203.00, TP: 43, FP: 47, FN 185, TN 1258 ,Threshold: 0.35 
Threshold: 0.40 -> Profit: 101.00, TP: 25, FP: 33, FN 203, TN 1272 ,Threshold: 0.40 
Threshold: 0.45 -> Profit: 106.00, TP: 20, FP: 18, FN 208, TN 1287 ,Threshold: 0.45 
Threshold: 0.50 -> Profit: 75.00, TP: 12, FP: 7, FN 216, TN 1298 ,Threshold: 0.50 
Threshold: 0.55 -> Profit: 25.00, TP: 5, FP: 5, FN 223, TN 1300 ,Threshold: 0.55 
Threshold: 0.60 -> Profit: -12.00, TP: 0, FP: 4, FN 228, TN 1301 ,Thre

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


{'Best Threshold': 0.25000000000000006,
 'Best profit on training set': 229,
 'Best profit on test set': 109,
 'Confusion matrix of training set': (1176, 129, 151, 77),
 'Confusion matrix of test set': (380, 49, 51, 32)}

# Logistic Regression Tuned and Scaled

In [ ]:
logistic_regression_tuned = LogisticRegression(random_state=42, class_weight = "balanced")
param_grid = {
    'penalty': ['l2'], #Only using l2, since this is the one compatible with lfgbs solver
    'C': [0.01, 0.1, 1, 10, 100],
    'max_iter': [500, 1000, 2000,3000]
}
grid_search = GridSearchCV(logistic_regression_tuned, param_grid, cv=5, scoring='f1')

grid_search.fit(X_train_scaled, y_train)

# Get the best parameters and model
best_logistic_regression_model = grid_search.best_estimator_
print("Best Parameters:", grid_search.best_params_)

# Use the best model to make predictions on the test set
y_pred = best_logistic_regression_model.predict(X_test_scaled)

training_score = best_logistic_regression_model.score(X_train_scaled, y_train)
print("Accuracy on training set: ", training_score)
print(f"Accuracy on test set: {metrics.accuracy_score(y_test, y_pred)} \n")

# Evaluate thresholds for the best model
evaluate_thresholds(best_logistic_regression_model, X_train_scaled, y_train, X_test_scaled, y_test)

Best Parameters: {'C': 1, 'max_iter': 500, 'penalty': 'l2'}
Accuracy on training set:  0.8343118069145466
Accuracy on test set: 0.810546875 

Threshold: 0.10 -> Profit: -842.00, TP: 227, FP: 886, FN 1, TN 419 ,Threshold: 0.10 
Threshold: 0.15 -> Profit: -417.00, TP: 225, FP: 739, FN 3, TN 566 ,Threshold: 0.15 
Threshold: 0.20 -> Profit: -75.00, TP: 222, FP: 617, FN 6, TN 688 ,Threshold: 0.20 
Threshold: 0.25 -> Profit: 164.00, TP: 217, FP: 524, FN 11, TN 781 ,Threshold: 0.25 
Threshold: 0.30 -> Profit: 387.00, TP: 216, FP: 447, FN 12, TN 858 ,Threshold: 0.30 
Threshold: 0.35 -> Profit: 507.00, TP: 210, FP: 391, FN 18, TN 914 ,Threshold: 0.35 
Threshold: 0.40 -> Profit: 650.00, TP: 205, FP: 330, FN 23, TN 975 ,Threshold: 0.40 
Threshold: 0.45 -> Profit: 762.00, TP: 201, FP: 282, FN 27, TN 1023 ,Threshold: 0.45 
Threshold: 0.50 -> Profit: 872.00, TP: 190, FP: 216, FN 38, TN 1089 ,Threshold: 0.50 
Threshold: 0.55 -> Profit: 903.00, TP: 180, FP: 179, FN 48, TN 1126 ,Threshold: 0.55 
Thresh

{'Best Threshold': 0.5500000000000002,
 'Best profit on training set': 903,
 'Best profit on test set': 316,
 'Confusion matrix of training set': (1126, 179, 48, 180),
 'Confusion matrix of test set': (361, 68, 18, 65)}

#KNN Without Tuning

In [ ]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

print(f"Accuracy on training set: {knn.score(X_train, y_train)}")
y_pred = knn.predict(X_test)
print(f"Accuracy on test set: {metrics.accuracy_score(y_test, y_pred)}\n")
evaluate_thresholds(knn, X_train, y_train, X_test, y_test)

Accuracy on training set: 0.8571428571428571
Accuracy on test set: 0.818359375

Threshold: 0.10 -> Profit: 186.00, TP: 228, FP: 546, FN 0, TN 759 ,Threshold: 0.10 
Threshold: 0.15 -> Profit: 186.00, TP: 228, FP: 546, FN 0, TN 759 ,Threshold: 0.15 
Threshold: 0.20 -> Profit: 572.00, TP: 130, FP: 156, FN 98, TN 1149 ,Threshold: 0.20 
Threshold: 0.25 -> Profit: 572.00, TP: 130, FP: 156, FN 98, TN 1149 ,Threshold: 0.25 
Threshold: 0.30 -> Profit: 572.00, TP: 130, FP: 156, FN 98, TN 1149 ,Threshold: 0.30 
Threshold: 0.35 -> Profit: 572.00, TP: 130, FP: 156, FN 98, TN 1149 ,Threshold: 0.35 
Threshold: 0.40 -> Profit: 227.00, TP: 40, FP: 31, FN 188, TN 1274 ,Threshold: 0.40 
Threshold: 0.45 -> Profit: 227.00, TP: 40, FP: 31, FN 188, TN 1274 ,Threshold: 0.45 
Threshold: 0.50 -> Profit: 227.00, TP: 40, FP: 31, FN 188, TN 1274 ,Threshold: 0.50 
Threshold: 0.55 -> Profit: 227.00, TP: 40, FP: 31, FN 188, TN 1274 ,Threshold: 0.55 
Threshold: 0.60 -> Profit: 48.00, TP: 6, FP: 0, FN 222, TN 1305 ,Thr

{'Best Threshold': 0.20000000000000004,
 'Best profit on training set': 572,
 'Best profit on test set': -38,
 'Confusion matrix of training set': (1149, 156, 98, 130),
 'Confusion matrix of test set': (371, 58, 66, 17)}

#KNN tuned and scaled

In [ ]:
knn_tuned = KNeighborsClassifier()

param_grid_knn = {
    'n_neighbors': [6,7,9,11,13,15,17],
    'p': [1, 2]
}

grid_search_knn = GridSearchCV(estimator=knn_tuned, param_grid=param_grid_knn, cv=5, scoring='accuracy', verbose=1)
grid_search_knn.fit(X_train_scaled, y_train)

# Get the best parameters and model
best_knn_model = grid_search_knn.best_estimator_
print("Best Parameters for KNN:", grid_search_knn.best_params_)

# Use the best model to make predictions on the scaled test set
y_pred_knn = best_knn_model.predict(X_test_scaled)

# Training and test accuracy
training_score_knn = best_knn_model.score(X_train_scaled, y_train)
print("Accuracy on training set:", training_score_knn)
print(f"Accuracy on test set: {metrics.accuracy_score(y_test, y_pred_knn)}")

evaluate_thresholds(best_knn_model, X_train_scaled, y_train, X_test_scaled, y_test)

Fitting 5 folds for each of 14 candidates, totalling 70 fits
Best Parameters for KNN: {'n_neighbors': 13, 'p': 1}
Accuracy on training set: 0.8767123287671232
Accuracy on test set: 0.865234375
Threshold: 0.10 -> Profit: 691.00, TP: 209, FP: 327, FN 19, TN 978 ,Threshold: 0.10 
Threshold: 0.15 -> Profit: 691.00, TP: 209, FP: 327, FN 19, TN 978 ,Threshold: 0.15 
Threshold: 0.20 -> Profit: 889.00, TP: 179, FP: 181, FN 49, TN 1124 ,Threshold: 0.20 
Threshold: 0.25 -> Profit: 825.00, TP: 138, FP: 93, FN 90, TN 1212 ,Threshold: 0.25 
Threshold: 0.30 -> Profit: 825.00, TP: 138, FP: 93, FN 90, TN 1212 ,Threshold: 0.30 
Threshold: 0.35 -> Profit: 687.00, TP: 105, FP: 51, FN 123, TN 1254 ,Threshold: 0.35 
Threshold: 0.40 -> Profit: 484.00, TP: 71, FP: 28, FN 157, TN 1277 ,Threshold: 0.40 
Threshold: 0.45 -> Profit: 484.00, TP: 71, FP: 28, FN 157, TN 1277 ,Threshold: 0.45 
Threshold: 0.50 -> Profit: 377.00, TP: 52, FP: 13, FN 176, TN 1292 ,Threshold: 0.50 
Threshold: 0.55 -> Profit: 302.00, TP: 4

{'Best Threshold': 0.20000000000000004,
 'Best profit on training set': 889,
 'Best profit on test set': 293,
 'Confusion matrix of training set': (1124, 181, 49, 179),
 'Confusion matrix of test set': (356, 73, 19, 64)}

#RandomForestClassifier Without Tuning

In [ ]:
rfc = RandomForestClassifier(random_state = 42)
rfc.fit(X_train, y_train)
print(f"Accuracy on training set: {rfc.score(X_train, y_train)}")
y_pred=rfc.predict(X_test)
print(f"Accuracy on test set: {metrics.accuracy_score(y_test, y_pred)}\n")
evaluate_thresholds(rfc, X_train, y_train, X_test, y_test)

Accuracy on training set: 0.9934768427919113
Accuracy on test set: 0.86328125

Threshold: 0.10 -> Profit: 1263.00, TP: 228, FP: 187, FN 0, TN 1118 ,Threshold: 0.10 
Threshold: 0.15 -> Profit: 1632.00, TP: 228, FP: 64, FN 0, TN 1241 ,Threshold: 0.15 
Threshold: 0.20 -> Profit: 1752.00, TP: 228, FP: 24, FN 0, TN 1281 ,Threshold: 0.20 
Threshold: 0.25 -> Profit: 1779.00, TP: 228, FP: 15, FN 0, TN 1290 ,Threshold: 0.25 
Threshold: 0.30 -> Profit: 1788.00, TP: 228, FP: 12, FN 0, TN 1293 ,Threshold: 0.30 
Threshold: 0.35 -> Profit: 1794.00, TP: 228, FP: 10, FN 0, TN 1295 ,Threshold: 0.35 
Threshold: 0.40 -> Profit: 1789.00, TP: 227, FP: 9, FN 1, TN 1296 ,Threshold: 0.40 
Threshold: 0.45 -> Profit: 1774.00, TP: 224, FP: 6, FN 4, TN 1299 ,Threshold: 0.45 
Threshold: 0.50 -> Profit: 1754.00, TP: 220, FP: 2, FN 8, TN 1303 ,Threshold: 0.50 
Threshold: 0.55 -> Profit: 1744.00, TP: 218, FP: 0, FN 10, TN 1305 ,Threshold: 0.55 
Threshold: 0.60 -> Profit: 1704.00, TP: 213, FP: 0, FN 15, TN 1305 ,Thres

{'Best Threshold': 0.3500000000000001,
 'Best profit on training set': 1794,
 'Best profit on test set': 303,
 'Confusion matrix of training set': (1295, 10, 0, 228),
 'Confusion matrix of test set': (394, 35, 32, 51)}

#RFC Tuned without scaling

In [ ]:
rfc_tuned = RandomForestClassifier(random_state=42, class_weight="balanced")

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3,4,5],
    #'criterion': ['gini', 'entropy'],        # Didn't improve performance -> taken out for less fits
    #'max_f#eatures': ['sqrt', 'log2'],       # Didn't improve performance -> taken out for less fits
    #'bootstrap': [True, False]               # Didn't improve performance -> taken out for less fits
}

grid_search = GridSearchCV(estimator=rfc_tuned, param_grid=param_grid, cv=5, scoring='f1', verbose=1)
grid_search.fit(X_train, y_train)

# Get the best parameters and model
best_decision_tree_model = grid_search.best_estimator_
print("Best Parameters:", grid_search.best_params_)

# Use the best model to make predictions on the test set
y_pred = best_decision_tree_model.predict(X_test)

training_score = best_decision_tree_model.score(X_train, y_train)
print("Accuracy on training set: ", training_score)
print(f"Accuracy on test set: {metrics.accuracy_score(y_test, y_pred)}")


evaluate_thresholds(best_decision_tree_model,  X_train, y_train, X_test, y_test)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
Best Parameters: {'max_depth': 5, 'n_estimators': 100}
Accuracy on training set:  0.8962818003913894
Accuracy on test set: 0.857421875
Threshold: 0.10 -> Profit: -1764.00, TP: 228, FP: 1196, FN 0, TN 109 ,Threshold: 0.10 
Threshold: 0.15 -> Profit: -1287.00, TP: 228, FP: 1037, FN 0, TN 268 ,Threshold: 0.15 
Threshold: 0.20 -> Profit: -819.00, TP: 228, FP: 881, FN 0, TN 424 ,Threshold: 0.20 
Threshold: 0.25 -> Profit: -269.00, TP: 227, FP: 695, FN 1, TN 610 ,Threshold: 0.25 
Threshold: 0.30 -> Profit: 121.00, TP: 227, FP: 565, FN 1, TN 740 ,Threshold: 0.30 
Threshold: 0.35 -> Profit: 470.00, TP: 220, FP: 430, FN 8, TN 875 ,Threshold: 0.35 
Threshold: 0.40 -> Profit: 797.00, TP: 217, FP: 313, FN 11, TN 992 ,Threshold: 0.40 
Threshold: 0.45 -> Profit: 989.00, TP: 202, FP: 209, FN 26, TN 1096 ,Threshold: 0.45 
Threshold: 0.50 -> Profit: 1127.00, TP: 184, FP: 115, FN 44, TN 1190 ,Threshold: 0.50 
Threshold: 0.55 -> Profit: 1150.00,

{'Best Threshold': 0.5500000000000002,
 'Best profit on training set': 1150,
 'Best profit on test set': 357,
 'Confusion matrix of training set': (1243, 62, 61, 167),
 'Confusion matrix of test set': (396, 33, 26, 57)}